In [1]:
from pathlib import Path
from typing import Dict, List, Any
import json
import re
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build, Resource
from googleapiclient.http import HttpRequest
from googleapiclient.errors import HttpError

In [2]:
path_token = Path("../api_key.json")
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
if not path_token.exists():
    raise FileNotFoundError(path_token)
with path_token.open(encoding='utf-8') as f:
    service_account_info = json.load(f)
creds = Credentials.from_service_account_info(service_account_info, scopes=SCOPES)
sheet_id = service_account_info['spreadsheet_id']

In [3]:
try:
    service: Resource = build('sheets', 'v4', credentials=creds)
    print(service)
    sheets: Resource = service.spreadsheets()
    sheet_info: List[Dict[str, Dict[str, str]]] = sheets.get(spreadsheetId=sheet_id, includeGridData=False, fields="sheets.properties(title)").execute()['sheets']
    sheets_name = [x['properties']['title'] for x in sheet_info]
    if 'Note' in sheets_name:
        sheets_name.remove('Note')
    
except HttpError as err:
    print(err)

In [4]:
response = sheets.get(
    spreadsheetId=sheet_id,
    ranges=[sheets_name[2]],
    includeGridData=True,
    fields="sheets/data/rowData/values/userEnteredValue,sheets/data/rowData/values/userEnteredFormat/backgroundColorStyle/rgbColor"
).execute()

In [5]:
regex_faq = re.compile(r"(?:https://https://arkhamfiles.github.io/)?(?:faq.html)?#?FAQ([0-9]+)")

In [6]:
### cell checking test
data = response['sheets'][0]['data'][0]
print(data.keys(), data['rowData'][0]['values'][0].keys(), '', sep='\n')
for i in [1, 2, 3, 5]:
    rowdata = data['rowData'][i]['values']
    print(f"row {i} with {rowdata[6]['userEnteredValue']}")
    if 'userEnteredFormat' in rowdata[6] and 'backgroundColorStyle' in rowdata[6]['userEnteredFormat']:
        bc = rowdata[6]['userEnteredFormat']['backgroundColorStyle']['rgbColor']
        if bc['red'] > bc['green'] * 1.05 and bc['red'] > bc['blue'] * 1.05:
            print("unnecessary row, color:", bc)
    if 'formulaValue' in rowdata[6]['userEnteredValue']:
        print("linked row, formular: ", rowdata[6]['userEnteredValue']['formulaValue'])
    else:
        match = regex_faq.search(rowdata[6]['userEnteredValue']['stringValue'])
        if match is not None:
            print("FAQ link request, FAQ number is", match.group(1))
    print()

dict_keys(['rowData'])
dict_keys(['userEnteredValue'])

row 1 with {'stringValue': '한국어판 미해당'}
unnecessary row, color: {'red': 0.95686275, 'green': 0.8, 'blue': 0.8}

row 2 with {'formulaValue': "='코'!G183"}
linked row, formular:  ='코'!G183

row 3 with {'stringValue': 'https://arkhamfiles.github.io/faq.html#FAQ40'}
FAQ link request, FAQ number is 40

row 5 with {'stringValue': '‘렉스 머피’는 덱에 <b><i>행운</b></i> 카드를 포함할 수 없습니다. 유의하세요.'}



In [15]:
key, value = next(iter(rowdata[6]['userEnteredValue'].items()))
print(len(rowdata[6]['userEnteredValue']))
print(key, value)

1
stringValue ‘렉스 머피’는 덱에 <b><i>행운</b></i> 카드를 포함할 수 없습니다. 유의하세요.


In [7]:
response = sheets.get(
    spreadsheetId=sheet_id,
    ranges=sheets_name,
    includeGridData=True,
    fields="sheets/data/rowData/values/userEnteredValue,sheets/data/rowData/values/userEnteredFormat/backgroundColorStyle/rgbColor"
).execute()

In [19]:
regex = re.compile(
    r"(?:https://arkhamfiles.github.io/)?([a-zA-Z]+).html(#[a-zA-Z0-9_]+)(%[0-9]+)?"
)

text = "https://arkhamfiles.github.io/faq.html#FAQ23"
match = regex.fullmatch(text)
print(match.groups() if match is not None else '')

text = "https://arkhamfiles.github.io/faq.html#FAQ23"
match = regex.fullmatch(text)
print(match.groups() if match is not None else '')

('faq', '#FAQ23', None)
